In [13]:
from torch import nn
from torch.nn import functional as F
class BasicBlock(nn.Sequential):
    def __init__(self,in_channels,out_channels,stride=1,downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1=nn.Conv2d(in_channels,out_channels,kernel_size=3,stride=stride,padding=1)
        self.bn1=nn.BatchNorm2d(out_channels)
        self.relu=nn.ReLU(inplace=True)
        self.conv2=nn.Conv2d(out_channels,out_channels,kernel_size=3,stride=1,padding=1)
        self.bn2=nn.BatchNorm2d(out_channels)
        self.downsample=downsample
    def forward(self,x):
        identity=x
        out=self.conv1(x)
        out=self.bn1(out)
        out=self.relu(out)
        out=self.conv2(out)
        out=self.bn2(out)
        if self.downsample is not None:
            identity=self.downsample(x)
        out=out+identity
        out=self.relu(out)
        return out











In [14]:
class ResNet(nn.Sequential):
    def __init__(self,block,num_blocks,num_classes=1000):
        super(ResNet, self).__init__()
        in_channels=3
        out_channels=64
        self.conv1=nn.Conv2d(in_channels=in_channels,out_channels=out_channels,kernel_size=7,stride=2,padding=3,bias=False)
        self.bn1=nn.BatchNorm2d(out_channels)
        self.relu=nn.ReLU(inplace=True)
        self.maxpool=nn.MaxPool2d(kernel_size=3,padding=1)
        self.layer1=self._build_layer(block,in_channels=64,out_channels=64,num_of_block=num_blocks[0],stride=1)
        self.layer2=self._build_layer(block,in_channels=64,out_channels=128,num_of_block=num_blocks[1],stride=2)
        self.layer3=self._build_layer(block,in_channels=128,out_channels=256,num_of_block=num_blocks[2],stride=2)
        self.layer4=self._build_layer(block,in_channels=256,out_channels=512,num_of_block=num_blocks[3],stride=2)
        self.avgpool=nn.AdaptiveAvgPool2d((1,1))
        self.fc=nn.Linear(512,num_classes)

    def _build_layer(self,block,in_channels,out_channels,num_of_block,stride=1):
        layers=[]
        downsample=None
        if in_channels!=out_channels or stride!=1:
            downsample=nn.Sequential(
            nn.Conv2d(in_channels=in_channels,out_channels=out_channels,kernel_size=1,stride=2),
            nn.BatchNorm2d(out_channels)
            )
        layers.append(block(in_channels=in_channels,out_channels=out_channels,stride=stride,downsample=downsample))
        for _ in range(num_of_block-1):
            layers.append(block(in_channels=out_channels,out_channels=out_channels,stride=1))

        return nn.Sequential(*layers)
    def forward(self,x):
        x=self.conv1(x)
        x=self.bn1(x)
        x=self.relu(x)
        x=self.maxpool(x)
        out=self.layer1(x)
        out=self.layer2(out)
        out=self.layer3(out)
        out=self.layer4(out)
        out=self.avgpool(out)
        out=torch.flatten(out,1)
        out=self.fc(out)

        return out








In [15]:
def resnet18():
  return ResNet(block=BasicBlock, num_blocks=[2,2,2,2])


In [24]:
resnet18=resnet18()

In [25]:
def resnet34():
    return ResNet(block=BasicBlock, num_blocks=[3,4,6,3])

In [26]:
resnet34=resnet34()

In [28]:
resnet18

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=3, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_s

In [29]:
resnet34

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=3, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_s

In [30]:
import torch
from torchsummary import summary
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
resnet34.to(device)
summary(resnet34, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 38, 38]               0
            Conv2d-5           [-1, 64, 38, 38]          36,928
       BatchNorm2d-6           [-1, 64, 38, 38]             128
              ReLU-7           [-1, 64, 38, 38]               0
            Conv2d-8           [-1, 64, 38, 38]          36,928
       BatchNorm2d-9           [-1, 64, 38, 38]             128
             ReLU-10           [-1, 64, 38, 38]               0
           Conv2d-11           [-1, 64, 38, 38]          36,928
      BatchNorm2d-12           [-1, 64, 38, 38]             128
             ReLU-13           [-1, 64, 38, 38]               0
           Conv2d-14           [-1, 64,

In [31]:
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, datasets

def dataloader_cifar():

    transform = transforms.Compose([transforms.ToTensor(),
                                    transforms.Resize(size=(224, 224)),
                                    transforms.Normalize(mean=[0.5], std=[0.5])])

    train_dataset = datasets.CIFAR10('/content/drive/MyDrive/All_Datasets/CIFAR10', train=True, download=True, transform=transform)
    test_dataset = datasets.CIFAR10('/content/drive/MyDrive/All_Datasets/CIFAR10', train=False, download=True, transform=transform)

    # Split dataset into training set and validation set.
    train_dataset, val_dataset = random_split(train_dataset, (45000, 5000))

    print("Image shape of a random sample image : {}".format(train_dataset[0][0].numpy().shape), end = '\n\n')

    print("Training Set:   {} images".format(len(train_dataset)))
    print("Validation Set:   {} images".format(len(val_dataset)))
    print("Test Set:       {} images".format(len(test_dataset)))

    BATCH_SIZE = 32

    # Generate dataloader
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

    return train_loader, val_loader, test_loader

In [32]:
train_loader, val_loader, test_loader = dataloader_cifar()

100%|██████████| 170M/170M [00:05<00:00, 31.7MB/s]


Extracting /content/drive/MyDrive/All_Datasets/CIFAR10/cifar-10-python.tar.gz to /content/drive/MyDrive/All_Datasets/CIFAR10
Files already downloaded and verified
Image shape of a random sample image : (3, 224, 224)

Training Set:   45000 images
Validation Set:   5000 images
Test Set:       10000 images


In [33]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet34.parameters(), lr=0.01)

In [40]:
def train_model():
    EPOCHS = 2
    train_samples_num = 45000
    val_samples_num = 5000
    train_costs, val_costs = [], []

    for epoch in range(EPOCHS):

        train_running_loss = 0
        correct_train = 0

        resnet34.train().cuda()

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            yhat = resnet34(inputs)
            loss = criterion(yhat, labels)
            loss.backward()
            optimizer.step()
            _, predicted_outputs = torch.max(yhat.data, 1)
            correct_train += (predicted_outputs == labels).float().sum().item()
            train_running_loss += (loss.data.item() * inputs.shape[0])

        train_epoch_loss = train_running_loss / train_samples_num
        train_costs.append(train_epoch_loss)
        train_acc =  correct_train / train_samples_num


        val_running_loss = 0
        correct_val = 0

        resnet34.eval().cuda()
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                yhat = resnet34(inputs)
                loss = criterion(yhat, labels)
                _, predicted_outputs = torch.max(yhat.data, 1)
                correct_val += (predicted_outputs == labels).float().sum().item()

            val_running_loss += (loss.data.item() * inputs.shape[0])
            val_epoch_loss = val_running_loss / val_samples_num
            val_costs.append(val_epoch_loss)
            val_acc =  correct_val / val_samples_num

        info = "[Epoch {}/{}]: train-loss = {:0.6f} | train-acc = {:0.3f} | val-loss = {:0.6f} | val-acc = {:0.3f}"
        print(info.format(epoch+1, EPOCHS, train_epoch_loss, train_acc, val_epoch_loss, val_acc))
        torch.save(resnet34.state_dict(), '/content/checkpoint_gpu_{}'.format(epoch + 1))

    torch.save(resnet34.state_dict(), '/content/resnet-34_weights_gpu')
    return train_costs, val_costs

In [ ]:
train_costs, val_costs = train_model()

In [ ]:
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model = resnet34()
model.load_state_dict(torch.load('/content/resnet-56_weights_gpu'))

In [ ]:

test_samples_num = 10000
correct = 0

model.eval().cuda()

with  torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        yhat = model(inputs)
        _, predicted_class = torch.max(yhat.data, 1)
        correct += (predicted_class == labels).float().sum().item()

test_accuracy = correct / test_samples_num
print('Test accuracy: {}'.format(test_accuracy))

torch.Size([32, 512, 1, 1])
torch.Size([32, 512, 1, 1])
torch.Size([32, 512, 1, 1])
torch.Size([32, 512, 1, 1])
torch.Size([32, 512, 1, 1])
torch.Size([32, 512, 1, 1])
torch.Size([32, 512, 1, 1])
torch.Size([32, 512, 1, 1])
torch.Size([32, 512, 1, 1])
torch.Size([32, 512, 1, 1])
torch.Size([32, 512, 1, 1])
torch.Size([32, 512, 1, 1])
torch.Size([32, 512, 1, 1])
torch.Size([32, 512, 1, 1])
torch.Size([32, 512, 1, 1])
torch.Size([32, 512, 1, 1])
torch.Size([32, 512, 1, 1])
torch.Size([32, 512, 1, 1])
torch.Size([32, 512, 1, 1])
torch.Size([32, 512, 1, 1])
torch.Size([32, 512, 1, 1])
torch.Size([32, 512, 1, 1])
torch.Size([32, 512, 1, 1])
torch.Size([32, 512, 1, 1])
torch.Size([32, 512, 1, 1])
torch.Size([32, 512, 1, 1])
torch.Size([32, 512, 1, 1])
torch.Size([32, 512, 1, 1])
torch.Size([32, 512, 1, 1])
torch.Size([32, 512, 1, 1])
torch.Size([32, 512, 1, 1])
torch.Size([32, 512, 1, 1])
torch.Size([32, 512, 1, 1])
torch.Size([32, 512, 1, 1])
torch.Size([32, 512, 1, 1])
torch.Size([32, 512,

In [ ]:
class BottleNeck(nn.Sequential):
  def __init__(self,in_channels,out_channels,stride=1,downsample=None):
    super(BottleNeck, self).__init__()
    self.conv1=nn.Conv2d(in_channels=in_channels,out_channels=out_channels//4,kernel_size=1,stride=1)
    self.bn1=nn.BatchNorm2d(out_channels//4)
    self.conv2=nn.Conv2d(in_channels=out_channels//4,out_channels=out_channels//4,kernel_size=3,stride=stride,padding=1)
    self.bn2=nn.BatchNorm2d(out_channels//4)
    self.conv3=nn.Conv2d(in_channels=out_channels//4,out_channels=out_channels,kernel_size=1,stride=1)
    self.bn3=nn.BatchNorm2d(out_channels)
    self.relu=nn.ReLU(inplace=True)
    self.downsample=downsample

  def forward(self,x):
    identity=x
    out=self.conv1(x)
    out=self.bn1(out)
    out=self.relu(out)
    out=self.conv2(out)
    out=self.bn2(out)
    out=self.relu(out)
    out=self.conv3(out)
    out=self.bn3(out)
    if self.downsample is not None:
      identity=self.downsample(x)
    out+=out+identity
    out=self.relu(out)
    return out

In [ ]:
class ResNet50(nn.Sequential):
  def __init__(self,block,num_blocks,num_classes=1000):
    super(ResNet50, self).__init__()
    in_channels=3
    out_channels=64
    self.conv0=nn.Conv2d(in_channels=in_channels,out_channels=out_channels,kernel_size=7,stride=2,padding=3,bias=False)
    self.bn0=nn.BatchNorm2d(out_channels)
    self.relu=nn.ReLU(inplace=True)
    self.maxpool=nn.MaxPool2d(kernel_size=3,stride=2,padding=1)
    self.layer1=self._build_layer(block,in_channels=64,out_channels=256,num_of_block=num_blocks[0],stride=1)
    self.layer2=self._build_layer(block,in_channels=256,out_channels=512,num_of_block=num_blocks[1],stride=2)
    self.layer3=self._build_layer(block,in_channels=512,out_channels=1024,num_of_block=num_blocks[2],stride=2)
    self.layer4=self._build_layer(block,in_channels=1024,out_channels=2048,num_of_block=num_blocks[3],stride=2)
    self.avgpool=nn.AdaptiveAvgPool2d((1,1))
    self.fc=nn.Linear(2048,num_classes)


  def _build_layer(self,block,in_channels,out_channels,num_of_block,stride):
    layers=[]
    downsample=None
    if in_channels!=out_channels or stride!=1:
      downsample=nn.Sequential(
      nn.Conv2d(in_channels=in_channels,out_channels=out_channels,kernel_size=1,stride=stride),
      nn.BatchNorm2d(out_channels)
      )
    layers.append(block(in_channels=in_channels,out_channels=out_channels,stride=stride,downsample=downsample))

    for _ in range(num_of_block-1):
      layers.append(block(in_channels=out_channels,out_channels=out_channels,stride=1))

    return nn.Sequential(*layers)
  def forward(self,x):
    x=self.conv0(x)
    x=self.bn0(x)
    x=self.relu(x)
    x=self.maxpool(x)
    x=self.layer1(x)
    x=self.layer2(x)
    x=self.layer3(x)
    x=self.layer4(x)
    x=self.avgpool(x)
    x=torch.flatten(x,1)
    x=self.fc(x)
    return x




In [ ]:
ResNet50(BottleNeck,num_blocks=[3,4,6,3])

ResNet50(
  (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BottleNeck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
        (1): BatchNorm2d(256, 

In [ ]:
def resnet50():
  return ResNet50(block=BottleNeck,num_blocks=[3,4,6,3])

In [ ]:
model=resnet50()

In [ ]:
import torch
from torchsummary import summary
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,160
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,928
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,640
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,640
      BatchNorm2d-14          [-1, 256,